In [1]:
import os

from datasets import load_dataset, load_from_disk

# Paths
cache_dir = "./cache"
processed_path = "./processed/alpaca-ptbr"

# Check if the processed dataset already exists
if os.path.exists(processed_path):
    print("📂 Loading dataset from local disk...")
    ds = load_from_disk(processed_path)
else:
    print("⬇️ Downloading and caching dataset...")
    ds = load_dataset("dominguesm/alpaca-data-pt-br", cache_dir=cache_dir)
    print("💾 Saving processed dataset to disk...")
    ds["train"].save_to_disk(processed_path)

# ds is ready to use
print(ds)

📂 Loading dataset from local disk...
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 51759
})


In [2]:
from transformers import AutoTokenizer

# Example usage of the GPT model
tokenizer = AutoTokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")

In [3]:
try:
    df = ds["train"].to_pandas()
except:
    df = ds.to_pandas()

In [4]:
df["concat"] = (
    df["instruction"] + "\n\n" + df["input"].fillna("") + "\n\n" + df["output"]
)

In [5]:
# create a column with count of tokens for each row
df["tokens"] = df["concat"].apply(lambda x: len(tokenizer.encode(x)))

In [6]:
df["tokens"].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

count    51759.000000
mean        74.283371
std         40.517703
min         10.000000
25%         38.000000
50%         67.000000
75%        110.000000
90%        127.000000
95%        136.000000
99%        159.000000
max        838.000000
Name: tokens, dtype: float64

## Train

In [ ]:
import torch

from model import GPT, GPTConfig

device = "cuda" if torch.cuda.is_available() else "cpu"

n_experts = 4
factor = 2
config = GPTConfig(
    block_size=128,
    vocab_size=tokenizer.vocab_size,
    n_layer=2 * factor,
    n_head=4 * factor,
    n_embd=256 * factor,
    n_experts=n_experts,
    capacity_factor=n_experts,
    k=2 * factor,
    experts_weight=1e-5,
    router_weight=1e-5,
    dropout=0.2,
    bias=True,
)

model = GPT(config)

number of parameters: 22.60M


In [8]:
inputs = tokenizer(
    df["concat"].tolist(),
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=128,
)

input_ids = inputs["input_ids"]
targets = input_ids.clone()

targets[:, :-1] = input_ids[:, 1:]
targets[:, -1] = -1

# convert targets 0 to -1
targets[targets == 0] = -1

# print("input ids:", inputs["input_ids"], "shape:", inputs["input_ids"].shape)

In [9]:
input_ids.shape

torch.Size([51759, 128])

In [10]:
# invert 0 -> 1 and 1 -> 0 in attention mask
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# create a new upper triangular mask
upper_mask = torch.triu(torch.ones(input_ids.shape[1], input_ids.shape[1]), diagonal=1)

# apply the mask to the attention mask
attention_mask = attention_mask.unsqueeze(1) * upper_mask.unsqueeze(0)

# conver to bool
attention_mask = attention_mask.bool()

In [11]:
input_ids.shape

torch.Size([51759, 128])

In [12]:
model(input_ids[:5, :], attention_mask=attention_mask[:5, :, :])[0].shape

Load: tensor([0.6406, 0.3812, 0.5031, 0.4750]), Importance: tensor([0.2790, 0.2314, 0.2461, 0.2436], grad_fn=<MeanBackward1>)
Router loss: 2.0012790628243238e-05, Balance loss: 5.064411652710987e-06
Load: tensor([0.6484, 0.5844, 0.3484, 0.4187]), Importance: tensor([0.2774, 0.2638, 0.2233, 0.2356], grad_fn=<MeanBackward1>)
Router loss: 2.0773921278305352e-05, Balance loss: 5.104488991491962e-06
Load: tensor([0.5641, 0.5750, 0.3781, 0.4828]), Importance: tensor([0.2641, 0.2578, 0.2327, 0.2454], grad_fn=<MeanBackward1>)
Router loss: 2.0520223188214004e-05, Balance loss: 5.036750735598616e-06
Load: tensor([0.4141, 0.6453, 0.4484, 0.4922]), Importance: tensor([0.2342, 0.2749, 0.2405, 0.2504], grad_fn=<MeanBackward1>)
Router loss: 2.038217098743189e-05, Balance loss: 5.054653229308315e-06


torch.Size([5, 1, 50257])

### Train loop

In [13]:
# send data to device
model = model.to(device)
input_ids = input_ids[:32, :].to(device)  # limit to 16 for testing
attention_mask = attention_mask[:32, :, :].to(device)
targets = targets[:32, :].to(device)

# get only one sample for each
input_ids = input_ids[:64, :]
attention_mask = attention_mask[:64, :, :]
targets = targets[:64, :]

In [14]:
input_ids.shape

torch.Size([32, 128])

In [15]:
from tqdm.notebook import tqdm

with torch.no_grad():
    logits, loss = model(input_ids, attention_mask, targets)
    # print("logits:", logits, "shape:", logits.shape)
print("loss:", loss.item() if loss is not None else "N/A")
if loss is not None:
    perplexity = torch.exp(loss)
    print("perplexity:", perplexity.item())

# pred_tokens = logits.argmax(dim=-1)
# tokens = pred_tokens[0].tolist()
# print("predicted tokens:", tokens)
# print("predicted text:", tokenizer.decode(tokens))

optimizer = model.configure_optimizers(
    weight_decay=0.0,
    learning_rate=3e-4,
    betas=(0.9, 0.95),
    device_type="cuda",
)

pbar = tqdm(range(450), desc="Training Epochs")

for epoch in pbar:
    model.train()
    optimizer.zero_grad()
    logits, loss = model(input_ids, attention_mask, targets)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        logits, loss = model(input_ids, attention_mask, targets)

    perplexity = torch.exp(loss)
    pbar.set_postfix(loss=loss.item(), perplexity=perplexity.item())

    pred_tokens = logits.argmax(dim=-1)
    tokens = pred_tokens[0].tolist()

# Router loss: 0.0004672443028539419, Balance loss: 0.019999997690320015

Load: tensor([0.6416, 0.3650, 0.4729, 0.5205], device='cuda:0'), Importance: tensor([0.2793, 0.2300, 0.2419, 0.2488], device='cuda:0')
Router loss: 2.0275301721994765e-05, Balance loss: 5.070401130069513e-06
Load: tensor([0.6643, 0.5408, 0.3694, 0.4255], device='cuda:0'), Importance: tensor([0.2813, 0.2553, 0.2266, 0.2367], device='cuda:0')
Router loss: 2.0866178601863794e-05, Balance loss: 5.094051630294416e-06
Load: tensor([0.5691, 0.5190, 0.4092, 0.5027], device='cuda:0'), Importance: tensor([0.2641, 0.2520, 0.2344, 0.2495], device='cuda:0')
Router loss: 2.0700686945929192e-05, Balance loss: 5.024237907491624e-06
Load: tensor([0.3921, 0.6772, 0.4468, 0.4839], device='cuda:0'), Importance: tensor([0.2320, 0.2802, 0.2400, 0.2478], device='cuda:0')
Router loss: 2.048265014309436e-05, Balance loss: 5.07871027366491e-06
loss: 10.85292911529541
perplexity: 51685.32421875
using fused AdamW: True


Training Epochs:   0%|          | 0/450 [00:00<?, ?it/s]

Load: tensor([0.6304, 0.3696, 0.4595, 0.5405], device='cuda:0'), Importance: tensor([0.2776, 0.2297, 0.2421, 0.2506], device='cuda:0',
       grad_fn=<MeanBackward1>)
Router loss: 2.0158677216386423e-05, Balance loss: 5.065863661002368e-06
Load: tensor([0.6775, 0.5386, 0.3579, 0.4260], device='cuda:0'), Importance: tensor([0.2829, 0.2558, 0.2265, 0.2348], device='cuda:0',
       grad_fn=<MeanBackward1>)
Router loss: 2.079398291243706e-05, Balance loss: 5.105296168039786e-06
Load: tensor([0.5493, 0.5317, 0.4082, 0.5107], device='cuda:0'), Importance: tensor([0.2614, 0.2533, 0.2339, 0.2514], device='cuda:0',
       grad_fn=<MeanBackward1>)
Router loss: 2.061144550680183e-05, Balance loss: 5.021592187404167e-06
Load: tensor([0.4097, 0.6914, 0.4302, 0.4688], device='cuda:0'), Importance: tensor([0.2341, 0.2806, 0.2398, 0.2454], device='cuda:0',
       grad_fn=<MeanBackward1>)
Router loss: 2.0577715986291878e-05, Balance loss: 5.081504241388757e-06
Load: tensor([0.6487, 0.3379, 0.4177, 0.59

In [17]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids[:, :1].clone(),
        max_new_tokens=100,
        temperature=0.1,
        top_k=1,
        greedy=True,
    )
generated_text = tokenizer.decode(generated_ids[0].tolist())
_df = df.head(32)
_df["generated_text"] = [
    tokenizer.decode(generated_ids[i].tolist()) for i in range(len(generated_ids))
]
print(f"Generated text after epoch {epoch + 1}: {generated_text}")


Load: tensor([0.3750, 0.4375, 0.6562, 0.5312], device='cuda:0'), Importance: tensor([0.2449, 0.2471, 0.2529, 0.2550], device='cuda:0')
Router loss: 1.7648095308686607e-05, Balance loss: 5.014331236452563e-06
Load: tensor([0.4062, 0.5938, 0.4062, 0.5938], device='cuda:0'), Importance: tensor([0.2486, 0.2535, 0.2531, 0.2447], device='cuda:0')
Router loss: 1.2154066553193843e-06, Balance loss: 4.996782536181854e-06
Load: tensor([0.3750, 0.8125, 0.3438, 0.4688], device='cuda:0'), Importance: tensor([0.2317, 0.2761, 0.2447, 0.2475], device='cuda:0')
Router loss: 9.518062142888084e-07, Balance loss: 5.113342922413722e-06
Load: tensor([0.7188, 0.2500, 0.5000, 0.5312], device='cuda:0'), Importance: tensor([0.2627, 0.2272, 0.2550, 0.2551], device='cuda:0')
Router loss: 9.183911515719956e-07, Balance loss: 5.08640505358926e-06
Load: tensor([0.2969, 0.6094, 0.3750, 0.7188], device='cuda:0'), Importance: tensor([0.2425, 0.2573, 0.2404, 0.2598], device='cuda:0')
Router loss: 1.164393233921146e-05, 

C:\Users\mathe\AppData\Local\Temp\ipykernel_35704\3646376486.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df["generated_text"] = [


In [18]:
_df.head(32).style

,instruction,input,output,concat,tokens,generated_text
0,Dê três dicas para se manter saudável.,,1. Coma uma dieta equilibrada e certifique-se de incluir muitas frutas e vegetais. 2. Exercite-se regularmente para manter seu corpo ativo e forte. 3. Durma o suficiente e mantenha um horário de sono consistente.,Dê três dicas para se manter saudável. 1. Coma uma dieta equilibrada e certifique-se de incluir muitas frutas e vegetais. 2. Exercite-se regularmente para manter seu corpo ativo e forte. 3. Durma o suficiente e mantenha um horário de sono consistente.,62,Dê três dicas para se manter saudável. 1. Coma uma dieta equilibrada e certifique-se de incluir muitas frutas e vegetais. 2. Exercite-se regularmente para manter seu corpo ativo e forte. 3. Durma o suficiente e mantenha um horário de sono consistente. 2. Exercite-se regularmente para manter saudável. 2. Dur. 3. O Canal do Panamá é uma importante rota de navegação e serve como uma hidrovia internacional conectando o Mar do
1,Quais são as três cores primárias?,,"As três cores primárias são vermelho, azul e amarelo.","Quais são as três cores primárias? As três cores primárias são vermelho, azul e amarelo.",23,"Quais são as três cores primárias? As três cores primárias são vermelho, azul e amarelo. A batalha foi um átomo. Foi é igual a morte de seu pai, Carlos I. Ele foi coroado em 1661 e seu reinado durou até sua morte em 1685. Carlos II também foi um patrono das artes, trabalhando com alguns dos maiores pintores e escultores de sua época. Ele também incentivou o teatro e os esportes e tinha interesse em ciências que também ajudaram a astronomia e a história natural. Carlos"
2,Descreva a estrutura de um átomo.,,"Um átomo é composto de um núcleo, que contém prótons e nêutrons, cercados por elétrons que viajam em órbitas ao redor do núcleo. Os prótons e nêutrons têm uma carga positiva, enquanto os elétrons têm uma carga negativa, resultando em um átomo neutro geral. O número de cada partícula determina o número atômico e o tipo de átomo.","Descreva a estrutura de um átomo. Um átomo é composto de um núcleo, que contém prótons e nêutrons, cercados por elétrons que viajam em órbitas ao redor do núcleo. Os prótons e nêutrons têm uma carga positiva, enquanto os elétrons têm uma carga negativa, resultando em um átomo neutro geral. O número de cada partícula determina o número atômico e o tipo de átomo.",78,"Descreva a função de uma placa-mãe de computador Uma placa-mãe de computador é a placa de circuito principal em um computador. É a espinha dorsal do sistema, conectando todos os outros componentes, como o processador, RAM, armazenamento, placa gráfica e portas periféricas. Ele fornece vias de comunicação entre todos esses componentes, permitindo que eles interajam e trabalhem juntos. A placa-mãe é responsável pela regulação de energia e também abriga slots para placas de expansão, bem como"
3,Como podemos reduzir a poluição do ar?,,"Há várias maneiras de reduzir a poluição do ar, como a mudança para fontes de energia renováveis, incentivando o uso de transporte público, proibindo a queima de combustíveis fósseis, implementando políticas para reduzir as emissões de fontes industriais e implementando padrões de emissões de veículos. Além disso, os indivíduos podem fazer sua parte para reduzir a poluição do ar, reduzindo o uso de carros, evitando a queima de materiais como madeira e mudando para aparelhos com eficiência energética.","Como podemos reduzir a poluição do ar? Há várias maneiras de reduzir a poluição do ar, como a mudança para fontes de energia renováveis, incentivando o uso de transporte público, proibindo a queima de combustíveis fósseis, implementando políticas para reduzir as emissões de fontes industriais e implementando padrões de emissões de veículos. Além disso, os indivíduos podem fazer sua parte para reduzir a poluição do ar, reduzindo o uso de carros, evitando a queima de materiais como madeira e mudando para aparelhos com eficiência energética.",101,"Como pode